In [11]:
import torch



class QSGDCompressor():

    def __init__(self ):
        super().__init__()
        self.quantum_num = 64

    def compress(self, tensor):
        #tensor = tensor.flatten()

        norm = tensor.norm()

        norm = norm.flatten()
        abs_gradient = tensor.abs()
        #print(norm)
        #print(abs_gradient)
        level_float = self.quantum_num / norm * abs_gradient
        #print(level_float)
        previous_level = level_float.floor()
        #x = torch.empty_like(torch.Tensor([1,2,3,4,5,67,7]))
        #print(x.uniform_())
        prob = torch.empty_like(tensor).uniform_()
        is_next_level = (prob < (level_float - previous_level))
        new_level = (previous_level + is_next_level)

        sign = tensor.sign()
        tensor_compressed = (new_level * sign)
        tensor_compressed = tensor_compressed
        tensor_compressed = tensor_compressed, norm

        return tensor_compressed

    def decompress(self, tensor_compressed):
        tensor_compressed, norm = tensor_compressed

        decode_output = tensor_compressed.type(torch.float32)
        tensor_decompressed = norm / self.quantum_num * decode_output
        #tensor_decompressed = tensor_decompressed.view(shape)
        return tensor_decompressed

x= torch.tensor([ 655665860.0,655665856.0,65566586034.0])
#print(x.uniform_())
#x= x.type(torch.float32)
a = QSGDCompressor()

y = a.compress(x)
print(y)
print("compressed")
y= a.decompress(y)
print(y)

(tensor([  1.,   1., 128.]), tensor([6.5573e+10]))
compressed
tensor([5.1229e+08, 5.1229e+08, 6.5573e+10])


In [ ]:
a=3
b=4
c= a<b
x = c+1
print(x)

In [ ]:
import torch



def sparsify(tensor, compress_ratio):
    tensor = tensor.flatten()
    k = max(1, int(tensor.numel() * compress_ratio))
    _, indices = torch.topk(tensor.abs(), k, sorted=False,)
    values = torch.gather(tensor, 0, indices)
    return values, indices


def desparsify(tensors, numel):
    values, indices = tensors
    tensor_decompressed = torch.zeros(numel, dtype=values.dtype, layout=values.layout, device=values.device)
    tensor_decompressed.scatter_(0, indices, values)
    return tensor_decompressed


class TopKCompressor():

    def __init__(self, compress_ratio):
        super().__init__()
        self.compress_ratio = compress_ratio

    def compress(self, tensor):
        tensors = sparsify(tensor, self.compress_ratio)
        ctx = tensor.numel(), tensor.size()
        print(tensor.numel())
        return tensors, ctx

    def decompress(self, tensors, ctx):
        """Decompress by filling empty slots with zeros and reshape back using the original shape"""
        numel, shape = ctx
        tensor_decompressed = desparsify(tensors, numel)
        return tensor_decompressed.view(shape)
x= torch.tensor([[ 1,2,3],[4,5,6],[7,88,9],[6,4,5]])
print(x)
a = TopKCompressor(0.4)
#print(x)
y, shape = a.compress(x)
print(y)
print("decompress")
y= a.decompress(y,shape)
print(y)

In [ ]:
import torch
import sys

a = torch.zeros([1])
print(a)
print(sys.getsizeof(a.storage()))

tensor([0.])
76


In [ ]:
!cat /proc/cpuinfo
!cat /proc/meminfo